In [0]:
df = spark.read.csv("/Volumes/workspace/default/data/club_stats.csv", header=True, inferSchema=True)

df.show(5)
df.printSchema()


+-----------------+---------+-----+-------+------------+---------+--------------+----------+
|           Player|     Club|Goals|Assists|Yellow_Cards|Red_Cards|Matches_Played|  Position|
+-----------------+---------+-----+-------+------------+---------+--------------+----------+
|     Lionel Messi|      PSG|   30|     18|           3|        0|            32|   Forward|
|Cristiano Ronaldo| Al Nassr|   25|      4|           5|        1|            31|   Forward|
|  Kevin De Bruyne| Man City|   10|     22|           4|        0|            28|Midfielder|
|  Virgil van Dijk|Liverpool|    5|      2|           3|        0|            30|  Defender|
|    Kylian Mbappe|      PSG|   28|     10|           2|        0|            33|   Forward|
+-----------------+---------+-----+-------+------------+---------+--------------+----------+
only showing top 5 rows
root
 |-- Player: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Goals: integer (nullable = true)
 |-- Assists: integer 

In [0]:
# Top goal Scorers
df.orderBy(df.Goals.desc()).select("Player","Club","Goals").show()

+-----------------+----------+-----+
|           Player|      Club|Goals|
+-----------------+----------+-----+
|   Erling Haaland|  Man City|   32|
|     Lionel Messi|       PSG|   30|
|    Kylian Mbappe|       PSG|   28|
|Cristiano Ronaldo|  Al Nassr|   25|
|        Neymar Jr|       PSG|   15|
|  Bruno Fernandes|Man United|   12|
|  Kevin De Bruyne|  Man City|   10|
|  Virgil van Dijk| Liverpool|    5|
|         Casemiro|Man United|    4|
|     Sergio Ramos|       PSG|    3|
+-----------------+----------+-----+



In [0]:
# Top Assist Providers

df.orderBy(df.Assists.desc()).select("Player","Club","Assists").show()

+-----------------+----------+-------+
|           Player|      Club|Assists|
+-----------------+----------+-------+
|  Kevin De Bruyne|  Man City|     22|
|     Lionel Messi|       PSG|     18|
|        Neymar Jr|       PSG|     12|
|    Kylian Mbappe|       PSG|     10|
|  Bruno Fernandes|Man United|      9|
|   Erling Haaland|  Man City|      6|
|Cristiano Ronaldo|  Al Nassr|      4|
|         Casemiro|Man United|      3|
|  Virgil van Dijk| Liverpool|      2|
|     Sergio Ramos|       PSG|      1|
+-----------------+----------+-------+



In [0]:
# Aggressive players with red and yellow cards

from pyspark.sql.functions import col, expr
df.withColumn("Total_Cards",col("Yellow_Cards") + col("Red_Cards")) \
    .orderBy(col("Total_cards").desc()) \
    .select("Player", "Club","Total_Cards") \
    .show()

+-----------------+----------+-----------+
|           Player|      Club|Total_Cards|
+-----------------+----------+-----------+
|     Sergio Ramos|       PSG|         12|
|         Casemiro|Man United|          9|
|  Bruno Fernandes|Man United|          7|
|Cristiano Ronaldo|  Al Nassr|          6|
|        Neymar Jr|       PSG|          6|
|  Kevin De Bruyne|  Man City|          4|
|     Lionel Messi|       PSG|          3|
|   Erling Haaland|  Man City|          3|
|  Virgil van Dijk| Liverpool|          3|
|    Kylian Mbappe|       PSG|          2|
+-----------------+----------+-----------+



In [0]:
# Club with most Goals

df.groupBy("Club").sum("Goals").orderBy("sum(Goals)", ascending=False).show()

+----------+----------+
|      Club|sum(Goals)|
+----------+----------+
|       PSG|        76|
|  Man City|        42|
|  Al Nassr|        25|
|Man United|        16|
| Liverpool|         5|
+----------+----------+



In [0]:
#Player efficiency score
from pyspark.sql.functions import round

df.withColumn("Efficiency_Score",round((col("Goals")*4 + col("Assists") * 3 -col("Yellow_Cards") - col("Red_Cards") * 2) / col("Matches_Played"), 2)).orderBy("Efficiency_Score", ascending=False).select("Player", "Efficiency_Score").show()

+-----------------+----------------+
|           Player|Efficiency_Score|
+-----------------+----------------+
|     Lionel Messi|            5.34|
|   Erling Haaland|            4.61|
|    Kylian Mbappe|            4.24|
|  Kevin De Bruyne|            3.64|
|        Neymar Jr|            3.56|
|Cristiano Ronaldo|            3.39|
|  Bruno Fernandes|            2.13|
|  Virgil van Dijk|            0.77|
|         Casemiro|             0.5|
|     Sergio Ramos|            0.04|
+-----------------+----------------+



In [0]:
#Club performnace summary

df.groupBy("Club").agg(
    {
        "Goals":"sum","Assists":"sum","Yellow_Cards":"sum","Red_cards":"sum","Matches_Played": "sum"
    }
).orderBy("sum(Goals)", ascending=False).show()

+----------+----------+------------+-----------------+--------------+-------------------+
|      Club|sum(Goals)|sum(Assists)|sum(Yellow_Cards)|sum(Red_cards)|sum(Matches_Played)|
+----------+----------+------------+-----------------+--------------+-------------------+
|       PSG|        76|          41|               20|             3|                117|
|  Man City|        42|          28|                7|             0|                 59|
|  Al Nassr|        25|           4|                5|             1|                 31|
|Man United|        16|          12|               15|             1|                 62|
| Liverpool|         5|           2|                3|             0|                 30|
+----------+----------+------------+-----------------+--------------+-------------------+



In [0]:
# Position Based stats

df.groupBy("Position").avg("Goals", "Assists", "Yellow_Cards", "Red_Cards").show()

+----------+-----------------+------------------+-----------------+------------------+
|  Position|       avg(Goals)|      avg(Assists)|avg(Yellow_Cards)|    avg(Red_Cards)|
+----------+-----------------+------------------+-----------------+------------------+
|  Defender|              4.0|               1.5|              6.5|               1.0|
|   Forward|             26.0|              10.0|              3.6|               0.4|
|Midfielder|8.666666666666666|11.333333333333334|6.333333333333333|0.3333333333333333|
+----------+-----------------+------------------+-----------------+------------------+



In [0]:
#Top Consistent Players

df.withColumn("Goals_Per_Match", round(col("Goals") / col("Matches_Played"), 2)) \
    .orderBy("Goals_Per_match", ascending=False) \
    .select("Player", "Goals", "Matches_Played", "Goals_Per_Match") \
    .show()

+-----------------+-----+--------------+---------------+
|           Player|Goals|Matches_Played|Goals_Per_Match|
+-----------------+-----+--------------+---------------+
|   Erling Haaland|   32|            31|           1.03|
|     Lionel Messi|   30|            32|           0.94|
|    Kylian Mbappe|   28|            33|           0.85|
|Cristiano Ronaldo|   25|            31|           0.81|
|        Neymar Jr|   15|            25|            0.6|
|  Bruno Fernandes|   12|            32|           0.38|
|  Kevin De Bruyne|   10|            28|           0.36|
|  Virgil van Dijk|    5|            30|           0.17|
|         Casemiro|    4|            30|           0.13|
|     Sergio Ramos|    3|            27|           0.11|
+-----------------+-----+--------------+---------------+



In [0]:
#Cards per match ratio

df.withColumn("Aggressiveness_Score",
              round((col("Yellow_Cards") + col("Red_Cards"))  / col("Matches_Played"), 2)
    ).orderBy("Aggressiveness_score",ascending=False).select("Player", "Aggressiveness_Score").show()

+-----------------+--------------------+
|           Player|Aggressiveness_Score|
+-----------------+--------------------+
|     Sergio Ramos|                0.44|
|         Casemiro|                 0.3|
|        Neymar Jr|                0.24|
|  Bruno Fernandes|                0.22|
|Cristiano Ronaldo|                0.19|
|  Kevin De Bruyne|                0.14|
|  Virgil van Dijk|                 0.1|
|   Erling Haaland|                 0.1|
|     Lionel Messi|                0.09|
|    Kylian Mbappe|                0.06|
+-----------------+--------------------+

